In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import gensim
from gensim import corpora
import pyLDAvis.gensim
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JZ\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1. Get dataset

In [2]:
file_path = 'C:/temp/dataset/data/enron_emails.csv'
df = pd.read_csv(file_path, sep = ',')

2. Explore Data Analysis

In [3]:
df.head(2)

,Message-ID,From,To,Date,content,detail_content,Unnamed: 6
0,<8345058.1075840404046.JavaMail.evans@thyme>,('advdfeedback@investools.com'),('advdfeedback@investools.com'),1/29/2002 23:20,INVESTools Advisory\nA Free Digest of Trusted ...,investools advisory free digest trusted invest...,NaN
1,<1512159.1075863666797.JavaMail.evans@thyme>,('richard.sanders@enron.com'),('richard.sanders@enron.com'),9/20/2000 19:07,----- Forwarded by Richard B Sanders/HOU/ECT o...,forwarded richard b sanders hou ect pm justin ...,NaN


In [4]:
df.shape

(14156, 7)

In [5]:
df = df.loc[:, ['Message-ID', 'From', 'To', 'Date','detail_content']]

In [ ]:
df.head(2)

In [ ]:
df.isnull().sum()

In [6]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df.head()

In [7]:
df = df.reset_index(drop=True)


In [ ]:
df.head()

3. Clean data

In [ ]:
# Tokenization
#text =df.apply(lambda row: word_tokenize(row['detail_content']), axis=1)
#text = text.rstrip()
#text = re.sub(r'[^a-zA-Z])', ' ', text)

In [8]:
# Define stopwords to exclude
stop = set(stopwords.words('english'))
stop.update(("to", "cc", "subject", "http", "from", "sent", "etc", "u", "www", "com"))

In [9]:
# Define punctuations to exclude
exclude = set(string.punctuation)

In [10]:
# Define lemmatizer from nltk
lemma = WordNetLemmatizer()

In [23]:

porter = PorterStemmer()

In [24]:
# Define word cleaning function
def clean(text, stop):
    text = text.rstrip()
    stop_free = " ".join([i for i in text.lower().split() if((i not in stop) and (not i.isdigit()))])
    punc_free = ''.join(i for i in stop_free if i not in exclude)
    normalized = " ".join(lemma.lemmatize(i) for i in punc_free.split())  
    cleaned_text = " ".join(porter.stem(token) for token in normalized.split())
    return cleaned_text

In [ ]:
#Stem words

cleaned_text = " ".join(porter.stem(token) for token in normalized.split())
print (cleaned_text)

In [25]:
# Clean the emails in df and print results
text_clean=[]
for text in df['detail_content']:
    text_clean.append(clean(text, stop).split())  
    #print(text_clean)

4. Prepare for model

In [26]:
# Define the dictionary
dictionary = corpora.Dictionary(text_clean)

In [27]:
print(dictionary)

Dictionary(20388 unique tokens: ['account', 'accur', 'acquir', 'acr', 'address']...)


In [28]:
# Define corpus
corpus = [dictionary.doc2bow(text) for text in (text_clean)]

In [29]:
print(corpus)

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 6), (6, 1), (7, 2), (8, 4), (9, 1), (10, 1), (11, 3), (12, 2), (13, 1), (14, 5), (15, 4), (16, 3), (17, 1), (18, 1), (19, 1), (20, 5), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 3), (29, 3), (30, 1), (31, 4), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 4), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 4), (53, 1), (54, 4), (55, 9), (56, 5), (57, 1), (58, 8), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 12), (79, 2), (80, 2), (81, 1), (82, 1), (83, 3), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 4), (92, 1), (93, 1), (94, 2), (95, 2), (96, 1), (97, 2), (98, 5), (99, 1), (100, 3), (101, 8), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1

5. Fit LDA model

In [37]:
# Define the LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, random_state=1,passes=5)

In [38]:
# Save the topics and top 5 words
topics = ldamodel.print_topics(num_words=5)

In [17]:
# Print the result
for topic in topics:
    print(topic)

(0, '0.017*"enron" + 0.011*"please" + 0.010*"thanks" + 0.009*"e" + 0.008*"know"')
(1, '0.036*"enron" + 0.018*"ect" + 0.008*"hou" + 0.007*"company" + 0.005*"energy"')
(2, '0.047*"td" + 0.037*"net" + 0.035*"money" + 0.032*"tr" + 0.029*"width"')
(3, '0.026*"enron" + 0.013*"message" + 0.010*"original" + 0.009*"development" + 0.008*"wj"')
(4, '0.047*"image" + 0.012*"click" + 0.012*"se" + 0.011*"ne" + 0.011*"sp"')


In [ ]:
6. Visualization of model 

In [39]:
# Prepare model for visualizatioon
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)


In [40]:
# display the topics
pyLDAvis.display(lda_display)


7. Get Dominant Topic with score

In [41]:
# Define function that retrieves topic details per row 
def get_topic_details(ldamodel, corpus):
    topic_details_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                topic_details_df = topic_details_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
    topic_details_df.columns = ['Dominant_Topic', '% Score', 'Topic_Keywords']
    return topic_details_df

In [42]:
# Concatenate original text with the outpu of the model detail
contents = pd.DataFrame({'Original Text': text_clean})
topic_details = pd.concat([get_topic_details(ldamodel, corpus), contents], axis=1)

In [43]:
topic_details.head(20)

,Dominant_Topic,% Score,Topic_Keywords,Original Text
0,0.0,0.9153,"enron, compani, energi, market, employe, time,...","[investool, advisori, free, digest, trust, inv..."
1,1.0,0.9622,"enron, ect, messag, hou, thank, pleas, pm, ori...","[forward, richard, b, sander, hou, ect, pm, ju..."
2,1.0,0.9784,"enron, ect, messag, hou, thank, pleas, pm, ori...","[hey, wear, target, purpl, shirt, today, mine,..."
3,0.0,0.9936,"enron, compani, energi, market, employe, time,...","[lesli, milosevich, santa, clara, avenu, alame..."
4,0.0,0.9942,"enron, compani, energi, market, employe, time,...","[rini, twait, e, th, ave, longmont, co, rtwait..."
5,0.0,0.9942,"enron, compani, energi, market, employe, time,...","[laurent, meillier, jackson, stq, oakland, ca,..."
6,1.0,0.7936,"enron, ect, messag, hou, thank, pleas, pm, ori...","[dear, mr, whalley, hope, take, time, read, co..."
7,0.0,0.9941,"enron, compani, energi, market, employe, time,...","[jan, nissl, e, state, bois, id, jnissl, healt..."
8,1.0,0.9905,"enron, ect, messag, hou, thank, pleas, pm, ori...","[much, nice, machin, origin, messag, baumbach,..."
9,0.0,0.9942,"enron, compani, energi, market, employe, time,...","[leigh, clemon, seyburn, drive, baton, roug, l..."
